# i.MXRT105x Flashloader

This example demonstrates how to create a flashloader for the i.MXRT105x device.

The Jupyter notebook guides you through the process of creating a High Assurance Boot (HAB) container with a flashloader application from the NXP SDK, which can be loaded using serial downloader mode.

The process is divided into several steps, outlined in the following sections.

> **Note**: This example is adaptable to other NXP devices that support HAB containers, such as i.MX RT117x and similar chip families.

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)

Let's prepare also workspace and variables.

In [1]:
# Initialization cell
import os

from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
%env JUPYTER_SPSDK=1
# Set a magic for command execution and echo
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [2]:
WORKSPACE = "workspace/"  # change this to path to your workspace
INPUTS = "inputs/"
# verbosity of commands, might be -v or -vv for debug or blank for no additional info
VERBOSITY = "-v"
UART_CONNECTION = "-p COM1"  # change this based on your comport
PORT = UART_CONNECTION.split(" ")[1]
HAB_USER_CONFIG = INPUTS + "rt105x_hab.yaml"
HAB_OUTPUT = WORKSPACE + "flashloader.bin"

## 2. Compiling the Flashloader Application

To begin, we need to build the flashloader application from the NXP SDK. Follow these steps:

 - Obtain the NXP MCUXpresso SDK by downloading it from the [NXP website](https://mcuxpresso.nxp.com/).
 - Locate the flashloader application in the SDK. It's typically found at: `{SDK_root}\boards\evkmimxrt1050\bootloader_examples\flashloader\cm4`
 - Use your preferred Integrated Development Environment (IDE) to open the flashloader application.
 - Build the flashloader and generate a binary output file.

> **Note**:  For this example, we're using a pre-compiled [flashloader](./inputs/rt105x_flashloader.bin) for convenience.


## 3. Preparing the HAB configuration file

Let's begin with creating a template configuration file using the `nxpimage hab get-template` command. This command generates a YAML template that can be customized for our specific HAB configuration needs. Below, we'll compare the differences between the template and our customized example to highlight the additions we've made.

In [3]:
# Get difference of template and user YAML configuration
YamlDiffWidget("rt105x_flashloader.diffc").html

nxpimage hab get-template -f mimxrt1050 -o workspace/rt105x_template.yaml --force 
The template file workspace/rt105x_template.yaml has been created.


## 4. HAB Container generation

Now that we have established the necessary configuration for generating the HAB container with the flashloader application, let's proceed to create the HAB container.

In [6]:
%! nxpimage hab export -c $HAB_USER_CONFIG -o $HAB_OUTPUT

# Just check the HAB container has been properly created
assert os.path.exists(HAB_OUTPUT)

nxpimage hab export -c inputs/rt105x_hab.yaml -o workspace/flashloader.bin 
Success. (HAB container: workspace/flashloader.bin created.)


## 5. Flashloader loading

The final step involves loading the generated flashloader into the MCU and executing it. The most straightforward approach is to utilize the 'sdphost' application with the 'write-file' and 'jump-address' commands.


Ensure that the chip is in serial downloader mode before proceeding.

### 5.1 Using CLI

In [6]:
%! sdphost $UART_CONNECTION write-file 0x20001c00 $HAB_OUTPUT
%! sdphost $UART_CONNECTION jump-address 0x20001c00

sdphost -p COM63 write-file 0x20001c00 workspace/flashloader.bin 
Status (HAB mode) = 1450735702 (0x56787856) Hab Is Disabled (Unlocked).
Response status = 2290649224 (0x88888888) Write File Success.
sdphost -p COM63 jump-address 0x20001c00 
Status (HAB mode) = 1450735702 (0x56787856) Hab Is Disabled (Unlocked).


### 5.2 Using API

In [7]:
from spsdk.exceptions import SPSDKError
from spsdk.sdp.interfaces.uart import SdpUARTInterface
from spsdk.sdp.sdp import SDP
from spsdk.utils.misc import load_binary

interfaces = SdpUARTInterface.scan(port=PORT, timeout=10000)
if not interfaces:
    raise SPSDKError("No SDP device found.")

flashloader = load_binary(HAB_OUTPUT)
with SDP(interfaces[0], True) as serial_downloader:
    serial_downloader.write_file(0x20001C00, flashloader)
    serial_downloader.jump_and_run(0x20001C00)
print("Flashloader has started.")

Flashloader has started.


## 6. Verification
Now it is time to verify that everything went well and the flashloader application has started. A new mboot device should be visible.

### 6.1 Using CLI

In [8]:
%! blhost $UART_CONNECTION get-property 1

blhost -p COM63 get-property 1 
Response status = 0 (0x0) Success.
Response word 1 = 1258424320 (0x4b020800)
Current Version = K2.8.0


### 6.2 Using API

In [9]:
from spsdk.mboot.interfaces.uart import MbootUARTInterface
from spsdk.mboot.mcuboot import McuBoot
from spsdk.mboot.properties import parse_property_value

interfaces = MbootUARTInterface.scan()
if not interfaces:
    raise SPSDKError("No mboot device found")

PROPERTY_INDEX = 1
with McuBoot(interfaces[0], True) as mb:
    property_raw = mb.get_property(PROPERTY_INDEX)
prop = parse_property_value(PROPERTY_INDEX, property_raw)
print(prop)

Current Version = K2.8.0
